# Analyzing RCT data with Precision Adjustment

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of six treatment groups. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data.
  

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

<ipython-input-2-c62976242921>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)


In [3]:
data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

,count
T4,
0,3354
1,1745


In [4]:
data.describe()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
count,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,...,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000
mean,10695.416356,1.368896,13.052952,12.281232,0.404001,0.121985,0.032555,0.007256,0.439694,0.012748,...,0.062954,0.110414,0.545009,0.109433,0.148068,0.109237,0.261032,0.218670,0.444205,0.342224
std,111.180503,1.898003,10.565602,10.362143,0.490746,0.327300,0.177487,0.084883,0.757622,0.112194,...,0.242903,0.313436,0.498019,0.312213,0.355202,0.311967,0.439240,0.413385,0.496926,0.474501
min,10404.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10600.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10698.000000,0.000000,11.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10796.000000,4.000000,25.000000,23.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,10880.000000,4.000000,52.000000,52.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Model
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicator,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, assuming the RCT assumptions hold. Within an RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However, we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.


We also consider an interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, assuming the RCT assumptions hold. Unlike the model without interactions, we are guaranteed to improve precision by considering the interactive model.

### Analysis

We consider

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

# Carry out covariate balance check


We first look at the coefficients individually with a $t$-test, and then we adjust the $p$-values to control for family-wise error.

The regression below is done using "type='HC1'" which computes the correct Eicker-Huber-White standard errors, instead of the classical non-robust formula based on homoscedasticity.

In [29]:
formula = ("0 ~ "
           "(female + black + othrace + C(dep) + q2 + q3 + q4 + q5 + q6 "
           "+ agelt35 + agegt54 + durable + lusd + husd)**2")
X = patsy.dmatrix(formula, data=data, return_type='dataframe')
X.shape

(5099, 120)

There is strong multicollinearity in the design matrix. In order to address this, we drop columns by thresholding using the QR decomposition.

In [6]:
def qr_decomposition(x):
    # Get a set of columns with no linear dependence for smallest subset of observations
    Q, Rx = np.linalg.qr(x)
    ex = np.abs(np.diag(Rx))
    keep = np.where(ex > 1e-6)[0]
    xS = x.iloc[:, keep]

    return xS


xS = qr_decomposition(X)
xS.shape

(5099, 103)

In [7]:
# individual t-tests
covariate_balance = sm.OLS(data["T4"], xS)
cb_fit = covariate_balance.fit(cov_type="HC1", use_t=True)
cb_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     T4   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     29.07
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        17:27:44   Log-Likelihood:                -3358.5
No. Observations:                5099   AIC:                             6923.
Df Residuals:                    4996   BIC:                             7596.
Df Model:                         102                                         
Covariance Type:                  HC1                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.3215      0.161      1.993      0.046       0.005       0.638
C(dep)[T.1]            -0.0736      0.201     -0.366      0.714      -0.468       0.320
C(dep)[T.2]            -0.1085      0.158     -0.689      0.491      -0.417       0.200
female                  0.1042      0.136      0.765      0.444      -0.163       0.371
female:C(dep)[T.1]      0.0555      0.046      1.194      0.233      -0.036       0.147
female:C(dep)[T.2]      0.0454      0.041      1.121      0.262      -0.034       0.125
black                   0.0716      0.083      0.864      0.387      -0.091       0.234
black:C(dep)[T.1]      -0.1173      0.068     -1.725      0.085      -0.251       0.016
black:C(dep)[T.2]      -0.0222      0.063     -0.350      0.726      -0.147       0.102
othrace                 0.0280      0.405      0.069      0.945      -0.766       0.822
othrace:C(dep)[T.1]    -0.8581      0.331     -2.594      0.010      -1.507      -0.210
othrace:C(dep)[T.2]     0.2415      0.133      1.812      0.070      -0.020       0.503
q2                     -0.0267      0.163     -0.164      0.870      -0.345       0.292
C(dep)[T.1]:q2          0.1658      0.201      0.825      0.409      -0.228       0.559
C(dep)[T.2]:q2          0.0875      0.157      0.559      0.576      -0.220       0.395
q3                     -0.0057      0.162     -0.035      0.972      -0.324       0.312
C(dep)[T.1]:q3          0.1210      0.200      0.606      0.545      -0.270       0.512
C(dep)[T.2]:q3          0.1396      0.157      0.891      0.373      -0.168       0.447
q4                      0.0433      0.162      0.267      0.789      -0.275       0.362
C(dep)[T.1]:q4          0.0848      0.199      0.426      0.670      -0.306       0.475
C(dep)[T.2]:q4          0.0910      0.157      0.580      0.562      -0.217       0.399
q5                      0.0939      0.162      0.581      0.561      -0.223       0.411
C(dep)[T.1]:q5          0.1105      0.198      0.557      0.578      -0.278       0.499
C(dep)[T.2]:q5          0.0960      0.155      0.618      0.537      -0.209       0.401
q6                     -0.2216      0.160     -1.386      0.166      -0.535       0.092
C(dep)[T.1]:q6          0.1007      0.209      0.482      0.630      -0.309       0.510
C(dep)[T.2]:q6          0.0484      0.164      0.295      0.768      -0.273       0.370
agelt35                -0.1092      0.133     -0.820      0.412      -0.370       0.152
C(dep)[T.1]:agelt35    -0.0737      0.049     -1.518      0.129      -0.169       0.021
C(dep)[T.2]:agelt35    -0.0244      0.038     -0.650      0.515      -0.098       0.049
agegt54                -0.4367      0.136     -3.215      0.001      -0.703      -0.170
C(dep)[T.1]:agegt54    -0.0806      0.068     -1.180      0.238      -0.214       0.053
C(d

To test balance conditions, we employ the Holm-Bonferroni step-down method. With 100+ hypotheses, the family-wise type I error, or the probability of making at least one type I error treating all hypotheses independently, is close to 1. To control for this, we adjust p-values with the following procedure.

First, set $\alpha=0.05$ and denote the list of $n$ p-values from the regression with the vector $p$.

1. Sort $p$ from smallest to largest, so $p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(n)}$. Denote the corresponding hypothesis for $p_{(i)}$ as $H_{(i)}$.
2. For $i=1,\ldots, n$,
- If $$p_{(i)} > \frac{\alpha}{n-i+1} $$ Break the loop and do not reject any $H_{(j)}$ for $j \geq i$.
- Else reject $H_{(i)}$ if $$p_{(i)} \leq \frac{\alpha}{n-i+1} $$ Increment $i := i+1$.




In [8]:
def holm_bonferroni(p, alpha=0.05):

    n = len(p)
    sig_beta = []

    for i in range(n):
        if np.sort(p)[i] > alpha / (n - i):
            break
        else:
            sig_beta.append(np.argsort(p)[i])
            i += 1

    return sig_beta


print("Significant Coefficients (Indices): ", holm_bonferroni(cb_fit.pvalues, alpha=0.05))

Significant Coefficients (Indices):  [83, 73, 93, 78]


<ipython-input-8-4a07ee4970a1>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig_beta.append(np.argsort(p)[i])
<ipython-input-8-4a07ee4970a1>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig_beta.append(np.argsort(p)[i])
<ipython-input-8-4a07ee4970a1>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig_beta.append(np.argsort(p)[i])
<ipython-input-8-4a07ee4970a1>:10: FutureWarning: Series.__getitem__ treating keys as posit

We see that that even though this is a randomized experiment, balance conditions are failed.

> Add blockquote


<!--
The holm method fails to reject any hypothesis. That is, we fail to reject the hypothesis that any coefficient is zero. Thus, in this randomized experiment, balance conditions are met. -->

# Model Specification

In [30]:
# no adjustment (2-sample approach)
cl = smf.ols("np.log(inuidur1) ~ T4", data)

# adding controls
cra = smf.ols("np.log(inuidur1) ~ T4 + "
              "(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
              data)
# Omitted dummies: q1, nondurable, muld

cl_results = cl.fit(cov_type="HC1")
cl_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.680
Date:                Sun, 26 Jan 2025   Prob (F-statistic):             0.0172
Time:                        18:22:45   Log-Likelihood:                -8223.8
No. Observations:                5099   AIC:                         1.645e+04
Df Residuals:                    5097   BIC:                         1.646e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0568      0.021     98.156      0.000       2.016       2.098
T4            -0.0855      0.036     -2.383      0.017      -0.156      -0.015
==============================================================================
Omnibus:                     2610.670   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              516.984
Skew:                          -0.542   Prob(JB):                    5.47e-113
Kurtosis:                       1.877   Cond. No.                         2.41
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [31]:
cra_results = cra.fit(cov_type="HC1")
cra_results.summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 120, but rank is 103
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     22.11
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:22:49   Log-Likelihood:                -8070.0
No. Observations:                5099   AIC:                         1.635e+04
Df Residuals:                    4995   BIC:                         1.703e+04
Df Model:                         103                                         
Covariance Type:                  HC1                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               2.2472      0.168     13.363      0.000       1.918       2.577
C(dep)[T.1]            -0.7197      0.581     -1.238      0.216      -1.859       0.419
C(dep)[T.2]            -0.0406      0.360     -0.113      0.910      -0.746       0.665
T4                     -0.0797      0.036     -2.239      0.025      -0.149      -0.010
female                 -0.1146      0.316     -0.362      0.717      -0.735       0.506
female:C(dep)[T.1]     -0.0279      0.118     -0.237      0.812      -0.258       0.202
female:C(dep)[T.2]      0.1482      0.103      1.440      0.150      -0.054       0.350
black                  -0.3895      0.106     -3.666      0.000      -0.598      -0.181
black:C(dep)[T.1]       0.1575      0.194      0.810      0.418      -0.224       0.539
black:C(dep)[T.2]      -0.1038      0.183     -0.567      0.571      -0.463       0.255
othrace                -1.1941      0.319     -3.749      0.000      -1.818      -0.570
othrace:C(dep)[T.1]     1.0736      0.641      1.674      0.094      -0.183       2.330
othrace:C(dep)[T.2]     0.0070      0.429      0.016      0.987      -0.834       0.848
q2                      0.2262      0.176      1.287      0.198      -0.118       0.571
C(dep)[T.1]:q2          0.6398      0.573      1.116      0.264      -0.484       1.763
C(dep)[T.2]:q2         -0.0445      0.358     -0.124      0.901      -0.747       0.658
q3                     -0.1540      0.175     -0.879      0.379      -0.497       0.189
C(dep)[T.1]:q3          0.6984      0.574      1.217      0.224      -0.426       1.823
C(dep)[T.2]:q3         -0.0295      0.359     -0.082      0.934      -0.733       0.674
q4                     -0.0475      0.177     -0.269      0.788      -0.394       0.299
C(dep)[T.1]:q4          0.5041      0.574      0.879      0.379      -0.620       1.628
C(dep)[T.2]:q4          0.1390      0.361      0.385      0.700      -0.569       0.847
q5                      0.0407      0.174      0.234      0.815      -0.301       0.382
C(dep)[T.1]:q5          0.5278      0.571      0.924      0.355      -0.591       1.647
C(dep)[T.2]:q5         -0.1697      0.359     -0.472      0.637      -0.874       0.535
q6                     -0.1078      0.220     -0.489      0.625      -0.540       0.324
C(dep)[T.1]:q6          1.0950      0.603      1.816      0.069      -0.087       2.277
C(dep)[T.2]:q6          0.3409      0.388      0.879      0.379      -0.419       1.101
agelt35                -0.6259      0.308     -2.035      0.042      -1.229      -0.023
C(dep)[T.1]:agelt35     0.0746      0.122      0.612      0.541      -0.164       0.314
C(dep)[T.2]:agelt35     0.0333      0.094      0.353      0.724      -0.152       0.218
agegt54                -0.3614      0.712     -0.507      0.612      -1.758       1.035
C(d

The interactive specification corresponds to the approach introduced in Lin (2013).

In [32]:
# interactive regression model

ira_formula = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])]  # clean column names
X = (X - X.mean(axis=0))  # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * (" + "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC1")
ira_results.summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 241, but rank is 197
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               inuidur1   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     26.53
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:23:01   Log-Likelihood:                -8016.3
No. Observations:                5099   AIC:                         1.644e+04
Df Residuals:                    4896   BIC:                         1.777e+04
Df Model:                         202                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0574      0.021     99.106      0.000       2.017       2.098
T4            -0.0752      0.036     -2.113      0.035      -0.145      -0.005
x0            -0.0826      0.291     -0.284      0.777      -0.653       0.488
x1            -0.0466      0.424     -0.110      0.912      -0.878       0.785
x2             0.1292      0.328      0.394      0.694      -0.514       0.772
x3            -0.6663      0.409     -1.629      0.103      -1.468       0.135
x4            -0.1734      0.142     -1.219      0.223      -0.452       0.105
x5             0.2166      0.127      1.711      0.087      -0.032       0.465
x6            -0.3928      0.127     -3.100      0.002      -0.641      -0.144
x7             0.1090      0.229      0.476      0.634      -0.340       0.558
x8            -0.2007      0.229     -0.876      0.381      -0.650       0.248
x9            -1.8729      0.598     -3.133      0.002      -3.045      -0.701
x10            2.5202      1.860      1.355      0.175      -1.124       6.165
x11            1.1889      0.575      2.069      0.039       0.063       2.315
x12           -0.0668      0.236     -0.283      0.777      -0.529       0.396
x13            0.2339      0.643      0.364      0.716      -1.027       1.494
x14           -0.2177      0.449     -0.485      0.628      -1.098       0.663
x15           -0.4334      0.236     -1.837      0.066      -0.896       0.029
x16            0.1543      0.645      0.239      0.811      -1.110       1.419
x17           -0.2545      0.451     -0.564      0.573      -1.139       0.630
x18           -0.2921      0.238     -1.230      0.219      -0.758       0.173
x19           -0.0239      0.644     -0.037      0.970      -1.286       1.238
x20           -0.1619      0.452     -0.358      0.720      -1.048       0.724
x21           -0.1841      0.235     -0.783      0.433      -0.645       0.276
x22            0.1396      0.640      0.218      0.827      -1.115       1.394
x23           -0.5626      0.450     -1.250      0.211      -1.445       0.319
x24           -0.4892      0.279     -1.754      0.079      -1.036       0.057
x25            0.4833      0.682      0.709      0.478      -0.853       1.820
x26           -0.0048      0.478     -0.010      0.992      -0.943       0.933
x27           -0.6777      0.410     -1.651      0.099      -1.482       0.127
x28            0.0508      0.150      0.339      0.734      -0.243       0.344
x29            0.1192      0.117      1.023      0.306      -0.109       0.348
x30           -0.3041      0.702     -0.433      0.665      -1.681       1.072
x31            0.0366      0.206      0.178      0.859      -0.367       0.440
x32            0.2231      0.422      0.529      0.597      -0.604       1.050
x33           -0.8380      0.598     -1.402      0.161      -2.010       0.334
x3

Next we try out partialling out with theoretically justified lasso. We use "plug-in" tuning with a theoretically valid choice of penalty $\lambda = 2 \cdot c \hat{\sigma} \sqrt{n} \Phi^{-1}(1-\alpha/2p)$, where $c>1$ and $1-\alpha$ is a confidence level, and $\Phi^{-1}$ denotes the quantile function. See book for more details.

We pull an analogue of R's rlasso. We find a Python code that tries to replicate the main function of hdm r-package. It was made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy). If not using colab, download its repository and copy this folder to your site-packages folder. In my case it is located here ***C:\Python\Python38\Lib\site-packages*** . We need to install this package ***pip install multiprocess***.

In [12]:
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.0/416.0 kB 8.2 MB/s eta 0:00:00
Cloning into 'hdmpy'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 70 (delta 39), reused 52 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 25.30 KiB | 3.61 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [33]:
import sys
sys.path.insert(1, "./hdmpy")

In [34]:
# We wrap the package so that it has the familiar sklearn API
import hdmpy
from sklearn.base import BaseEstimator


class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])


def lasso_model():
    return RLasso(post=False)

In [35]:
D = X['T4']
A = X.drop('T4', axis=1)  # contains X and interaction terms D*X
y = np.log(data["inuidur1"])

Dres = D - np.mean(D)
yres = y - lasso_model().fit(A, y).predict(A)

ira_lasso_results = sm.OLS(yres, Dres).fit(cov_type='HC1')
ira_lasso_results.summary()

/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encounter

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               inuidur1   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):              0.001
Method:                 Least Squares   F-statistic:                              4.130
Date:                Sun, 26 Jan 2025   Prob (F-statistic):                      0.0422
Time:                        18:23:15   Log-Likelihood:                         -8154.8
No. Observations:                5099   AIC:                                  1.631e+04
Df Residuals:                    5098   BIC:                                  1.632e+04
Df Model:                           1                                                  
Covariance Type:                  HC1                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
T4            -0.0718      0.035     -2.032      0.042      -0.141      -0.003
==============================================================================
Omnibus:                     2424.706   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              502.658
Skew:                          -0.532   Prob(JB):                    7.07e-110
Kurtosis:                       1.889   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity robust (HC1)
"""

### Results

In [36]:
results = {
    "CL": [cl_results.params[cl.exog_names.index("T4")],
           cl_results.HC1_se[cl.exog_names.index("T4")]],
    "CRA": [cra_results.params[cra.exog_names.index("T4")],
            cra_results.HC1_se[cra.exog_names.index("T4")]],
    "IRA": [ira_results.params[ira.exog_names.index("T4")],
            ira_results.HC1_se[ira.exog_names.index("T4")]],
    "IRA w Lasso": [ira_lasso_results.params[0],
                    ira_lasso_results.bse[0]],
}
results = pd.DataFrame(results)
results.index = ["Estimate", "Standard Error"]
results
# for printing to LaTeX: print(results.to_latex())

<ipython-input-36-f0aeb3791028>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CL": [cl_results.params[cl.exog_names.index("T4")],
<ipython-input-36-f0aeb3791028>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cl_results.HC1_se[cl.exog_names.index("T4")]],
<ipython-input-36-f0aeb3791028>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CRA": [cra_results.params[cra.exog_names.index("T4")],
<ipython-input-36-f0aeb3791028>:5: Fu

,CL,CRA,IRA,IRA w Lasso
Estimate,-0.085455,-0.079680,-0.075218,-0.071767
Standard Error,0.035856,0.035591,0.035604,0.035316


[link text](https://)Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From the IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.




## **Adjustments with more Flexible Controls to CRA, IRA, and IRA with Lasso Models**

CRA Model Adjustments

In [40]:
# adding controls with more flexible interactions to cra model to test impact
# Added third-order interactions to estimate effects of combinations of three variables at a time/capturing more complex interactions

cra_2 = smf.ols("np.log(inuidur1) ~ T4 + "
              "(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**3",
              data)

cra_2_results = cra_2.fit(cov_type="HC1")
cra_2_results.summary()


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 562, but rank is 281
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     380.9
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:24:21   Log-Likelihood:                -7959.0
No. Observations:                5099   AIC:                         1.657e+04
Df Residuals:                    4773   BIC:                         1.870e+04
Df Model:                         325                                         
Covariance Type:                  HC1                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       2.5691      0.140     18.382      0.000       2.295       2.843
C(dep)[T.1]                     0.8933      0.644      1.386      0.166      -0.369       2.156
C(dep)[T.2]                    -0.4723      0.242     -1.951      0.051      -0.947       0.002
T4                             -0.0834      0.037     -2.276      0.023      -0.155      -0.012
female                         -0.2427      0.232     -1.048      0.295      -0.697       0.211
female:C(dep)[T.1]             -3.1460      0.694     -4.535      0.000      -4.506      -1.786
female:C(dep)[T.2]              0.4833      0.755      0.640      0.522      -0.997       1.963
black                          -0.3288      0.161     -2.044      0.041      -0.644      -0.014
black:C(dep)[T.1]               0.2150      0.364      0.590      0.555      -0.499       0.929
black:C(dep)[T.2]              -0.2304      0.355     -0.649      0.516      -0.926       0.465
othrace                        -1.3025      0.216     -6.029      0.000      -1.726      -0.879
othrace:C(dep)[T.1]             1.6290      0.434      3.754      0.000       0.778       2.480
othrace:C(dep)[T.2]            -0.4523      0.257     -1.760      0.078      -0.956       0.051
q2                             -0.2699      0.169     -1.600      0.110      -0.601       0.061
C(dep)[T.1]:q2                 -0.9587      0.667     -1.438      0.150      -2.266       0.348
C(dep)[T.2]:q2                  0.4191      0.274      1.530      0.126      -0.118       0.956
q3                             -0.3791      0.173     -2.192      0.028      -0.718      -0.040
C(dep)[T.1]:q3                 -0.9791      0.660     -1.483      0.138      -2.273       0.315
C(dep)[T.2]:q3                  0.2765      0.275      1.005      0.315      -0.263       0.816
q4                             -0.4087      0.174     -2.344      0.019      -0.751      -0.067
C(dep)[T.1]:q4                 -0.9478      0.662     -1.431      0.152      -2.246       0.351
C(dep)[T.2]:q4                  0.5537      0.290      1.909      0.056      -0.015       1.122
q5                             -0.2693      0.169     -1.592      0.111      -0.601       0.062
C(dep)[T.1]:q5                 -1.1439      0.660     -1.733      0.083      -2.438       0.150
C(dep)[T.2]:q5                  0.2886      0.283      1.020      0.308      -0.266       0.843
q6                             -0.6173      0.327     -1.885      0.059      -1.259       0.024
C(dep)[T.1]:q6                 -0.3849      0.780     -0.493      0.622      -1.914       1.144
C(dep)[T.2]:q6                  0.8238      0.416      1.980      0.048       0.008       1.639
agelt35                        -0.8634      0.195     -4.424      0.000      -1.246      -0.481
C(dep)[T.1]

In [41]:
# adding controls with more flexible interactions to cra model to test impact
# added fourth-order interactions to estimate effects of combinations of three variables at a time/capturing more complex interactions
cra_3 = smf.ols("np.log(inuidur1) ~ T4 + "
              "(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**4",
              data)

cra_3_results = cra_3.fit(cov_type="HC1")
cra_3_results.summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1849, but rank is 449
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     47.68
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:24:53   Log-Likelihood:                -7834.0
No. Observations:                5099   AIC:                         1.678e+04
Df Residuals:                    4545   BIC:                         2.040e+04
Df Model:                         553                                         
Covariance Type:                  HC1                                         
=======================================================================================================
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               2.5982      0.139     18.713      0.000       2.326       2.870
C(dep)[T.1]                             0.3959      0.394      1.006      0.315      -0.376       1.168
C(dep)[T.2]                            -0.5999      0.229     -2.625      0.009      -1.048      -0.152
T4                                     -0.0823      0.038     -2.164      0.030      -0.157      -0.008
female                                 -0.3425      0.256     -1.340      0.180      -0.844       0.159
female:C(dep)[T.1]                     -0.9157      0.411     -2.230      0.026      -1.721      -0.111
female:C(dep)[T.2]                      0.8241      0.505      1.631      0.103      -0.166       1.815
black                                  -0.2865      0.193     -1.483      0.138      -0.665       0.092
black:C(dep)[T.1]                       0.1616      0.373      0.433      0.665      -0.569       0.892
black:C(dep)[T.2]                      -1.0554      0.542     -1.947      0.051      -2.118       0.007
othrace                                -1.2782      0.172     -7.422      0.000      -1.616      -0.941
othrace:C(dep)[T.1]                     0.3366      0.079      4.260      0.000       0.182       0.491
othrace:C(dep)[T.2]                    -0.3386      0.155     -2.178      0.029      -0.643      -0.034
q2                                     -0.1785      0.180     -0.995      0.320      -0.530       0.173
C(dep)[T.1]:q2                         -1.2249      0.547     -2.240      0.025      -2.297      -0.153
C(dep)[T.2]:q2                          0.5930      0.280      2.116      0.034       0.044       1.142
q3                                     -0.5368      0.190     -2.826      0.005      -0.909      -0.164
C(dep)[T.1]:q3                         -0.2833      0.495     -0.572      0.567      -1.254       0.688
C(dep)[T.2]:q3                          0.7039      0.303      2.323      0.020       0.110       1.298
q4                                     -0.3980      0.188     -2.120      0.034      -0.766      -0.030
C(dep)[T.1]:q4                         -0.2043      0.498     -0.410      0.682      -1.181       0.772
C(dep)[T.2]:q4                          0.3028      0.337      0.899      0.369      -0.357       0.963
q5                                     -0.3234      0.190     -1.700      0.089      -0.696       0.049
C(dep)[T.1]:q5                         -0.5118      0.500     -1.023      0.307      -1.493       0.469
C(dep)[T.2]:q5                          0.3208      0.320      1.001      0.317      -0.307       0.949
q6                                     -0.7251      0.504     -1.438      0.150      -1.713       0.263
C(dep)[T.1]:q6                          1.4284      0.702      2.03

IRA Model Adjustments

In [42]:
# adding more flexible controls to interactive regression model to test impact
# third-order interactions exceeded maximum recursion depth, so only performing third-order interactions on certain variables

ira_formula_2 = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female*black*C(dep))**3"
X_2 = patsy.dmatrix(ira_formula_2, data, return_type='dataframe')
X_2.columns = [f'x{t}' for t in range(X_2.shape[1])]  # clean column names
X_2 = (X_2 - X_2.mean(axis=0))  # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula_2 = "T4 * (" + "+".join(X_2.columns) + ")"
X_2['T4'] = data['T4']
X_2 = patsy.dmatrix(ira_formula_2, X_2, return_type='dataframe')
ira_2 = sm.OLS(np.log(data[["inuidur1"]]), X_2)
ira_2_results = ira_2.fit(cov_type="HC1")
ira_2_results.summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 245, but rank is 201
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               inuidur1   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     26.06
Date:                Sun, 26 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:25:00   Log-Likelihood:                -8015.2
No. Observations:                5099   AIC:                         1.644e+04
Df Residuals:                    4892   BIC:                         1.780e+04
Df Model:                         206                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0573      0.021     99.080      0.000       2.017       2.098
T4            -0.0749      0.036     -2.105      0.035      -0.145      -0.005
x0            -0.0747      0.292     -0.256      0.798      -0.647       0.497
x1            -0.0515      0.426     -0.121      0.904      -0.886       0.783
x2             0.1261      0.329      0.383      0.701      -0.519       0.771
x3            -0.6808      0.410     -1.662      0.096      -1.484       0.122
x4            -0.1405      0.151     -0.927      0.354      -0.437       0.156
x5             0.2614      0.131      2.000      0.045       0.005       0.518
x6            -0.4153      0.129     -3.211      0.001      -0.669      -0.162
x7             0.2452      0.330      0.742      0.458      -0.402       0.893
x8            -0.0068      0.307     -0.022      0.982      -0.609       0.596
x9            -1.8577      0.596     -3.117      0.002      -3.026      -0.690
x10            2.4991      1.858      1.345      0.179      -1.142       6.140
x11            1.1697      0.567      2.064      0.039       0.059       2.280
x12           -0.0640      0.236     -0.271      0.786      -0.526       0.398
x13            0.2304      0.645      0.357      0.721      -1.034       1.495
x14           -0.2208      0.450     -0.490      0.624      -1.104       0.662
x15           -0.4310      0.236     -1.828      0.068      -0.893       0.031
x16            0.1549      0.647      0.239      0.811      -1.113       1.423
x17           -0.2621      0.453     -0.579      0.563      -1.149       0.625
x18           -0.2898      0.237     -1.221      0.222      -0.755       0.175
x19           -0.0238      0.646     -0.037      0.971      -1.289       1.242
x20           -0.1640      0.453     -0.362      0.717      -1.052       0.724
x21           -0.1869      0.235     -0.796      0.426      -0.647       0.273
x22            0.1389      0.642      0.216      0.829      -1.119       1.396
x23           -0.5553      0.451     -1.231      0.218      -1.439       0.329
x24           -0.4926      0.278     -1.769      0.077      -1.038       0.053
x25            0.4824      0.684      0.706      0.480      -0.858       1.822
x26           -0.0024      0.479     -0.005      0.996      -0.941       0.936
x27           -0.6787      0.411     -1.653      0.098      -1.483       0.126
x28            0.0497      0.150      0.332      0.740      -0.244       0.343
x29            0.1210      0.116      1.039      0.299      -0.107       0.349
x30           -0.3020      0.700     -0.431      0.666      -1.675       1.070
x31            0.0405      0.207      0.196      0.845      -0.365       0.446
x32            0.1893      0.434      0.436      0.663      -0.662       1.041
x33           -0.8303      0.599     -1.386      0.166      -2.005       0.344
x3

IRA w/ Lasso

In [ ]:
import sys
sys.path.insert(1, "./hdmpy")

In [43]:
#using adjusted IRA model code for IRA w/ Lasso model
D_2 = X_2['T4']
A_2 = X_2.drop('T4', axis=1)  # contains X and interaction terms D*X
y_2 = np.log(data["inuidur1"])

Dres_2 = D_2 - np.mean(D_2)
yres_2 = y_2 - lasso_model().fit(A_2, y_2).predict(A_2)

ira_lasso_2_results = sm.OLS(yres_2, Dres_2).fit(cov_type='HC1')
ira_lasso_2_results.summary()

/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encounter

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               inuidur1   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):              0.001
Method:                 Least Squares   F-statistic:                              4.137
Date:                Sun, 26 Jan 2025   Prob (F-statistic):                      0.0420
Time:                        18:25:09   Log-Likelihood:                         -8155.0
No. Observations:                5099   AIC:                                  1.631e+04
Df Residuals:                    5098   BIC:                                  1.632e+04
Df Model:                           1                                                  
Covariance Type:                  HC1                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
T4            -0.0718      0.035     -2.034      0.042      -0.141      -0.003
==============================================================================
Omnibus:                     2425.229   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              502.701
Skew:                          -0.532   Prob(JB):                    6.92e-110
Kurtosis:                       1.889   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity robust (HC1)
"""

## **Results with Adjusted Models**

In [38]:
results_2 = {
    "CRA with Third-Order Interactions": [cra_2_results.params[cra_2.exog_names.index("T4")],
           cra_2_results.HC1_se[cra_2.exog_names.index("T4")]],
    "CRA with Fourth-Order Interactions": [cra_3_results.params[cra_3.exog_names.index("T4")],
            cra_3_results.HC1_se[cra_3.exog_names.index("T4")]],
    "IRA with some Third-Order Interactions": [ira_2_results.params[ira_2.exog_names.index("T4")],
            ira_2_results.HC1_se[ira_2.exog_names.index("T4")]],
    "IRA w Lasso with some Third-Order Interactions": [ira_lasso_2_results.params[0],
                    ira_lasso_2_results.bse[0]],
}
results_2 = pd.DataFrame(results_2)
results_2.index = ["Estimate", "Standard Error"]
results_2
# for printing to LaTeX: print(results.to_latex())

<ipython-input-38-6fcb79ed3617>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CRA with Third-Order Interactions": [cra_2_results.params[cra_2.exog_names.index("T4")],
<ipython-input-38-6fcb79ed3617>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cra_2_results.HC1_se[cra_2.exog_names.index("T4")]],
<ipython-input-38-6fcb79ed3617>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CRA with Fourth-Order Interactions": [cra_3_resul

,CRA with Third-Order Interactions,CRA with Fourth-Order Interactions,IRA with some Third-Order Interactions,IRA w Lasso with some Third-Order Interactions
Estimate,-0.083407,-0.082297,-0.074932,-0.071837
Standard Error,0.036641,0.038036,0.035605,0.035317


Re-running results from original models for easy comparison

In [44]:
results = {
    "CL": [cl_results.params[cl.exog_names.index("T4")],
           cl_results.HC1_se[cl.exog_names.index("T4")]],
    "CRA": [cra_results.params[cra.exog_names.index("T4")],
            cra_results.HC1_se[cra.exog_names.index("T4")]],
    "IRA": [ira_results.params[ira.exog_names.index("T4")],
            ira_results.HC1_se[ira.exog_names.index("T4")]],
    "IRA w Lasso": [ira_lasso_results.params[0],
                    ira_lasso_results.bse[0]],
}
results = pd.DataFrame(results)
results.index = ["Estimate", "Standard Error"]
results
# for printing to LaTeX: print(results.to_latex())

<ipython-input-44-f0aeb3791028>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CL": [cl_results.params[cl.exog_names.index("T4")],
<ipython-input-44-f0aeb3791028>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cl_results.HC1_se[cl.exog_names.index("T4")]],
<ipython-input-44-f0aeb3791028>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CRA": [cra_results.params[cra.exog_names.index("T4")],
<ipython-input-44-f0aeb3791028>:5: Fu

,CL,CRA,IRA,IRA w Lasso
Estimate,-0.085455,-0.079680,-0.075218,-0.071767
Standard Error,0.035856,0.035591,0.035604,0.035316


## **Findings**

For initial analysis on how the adjusted models with more flexible controls compare to the original models, I took a look at some statistical measures to determine which might indicate a better model fit. For the CRA models, the adjusted models using 3rd and 4th order interactions have higher r-squared values (higher proportion of the dependent variables variance is explained by independent variables), larger log-likelihoods (higher likelihood observed data could be generated by model), and higher F-statistic values (explains significant portion of variability in dependent variables). For the IRA model (without Lasso), the adjusted model has a higher r-squared value and larger log-likelihood. For the IRA model using Lasso, the adjusted model actually has a lower log-likelihood and the same R-squared value. This could be a result of overfitting the data or increased sensitivity to outliers due to its incresed complexity. All that to say, it appears for the most part the adjusted models are performing better than the original models and could be better at predicting dependent variable.

As to the actual results, Treatment group 4 experiences an average decrease of about $7.812\%$ in the length of unemployment spell as compared the original models average decrease of $7.803\%$. This slightly higher decrease can mostly be attributed to the increase in estimates from the CRA adjusted models (7.97% to about 8.3%)

Across the board we can observe that adjusted model regression estimators delivers estimates that are slighly less efficient (higher standard errors) than the original models. Again, this increase in standard error can mostly be attributed to the CRA adjusted models (3.5% in original models to about 3.7% in adjusted models). This could be a result of overfitting due to increased model complexity.